# 11장 심층신경망 훈련

* 심층신경망
    * 수백 개의 뉴련으로 구성된 10개 이상의 층을 사용하는 신경망
    * 예제: 고해상도 이미지에서 수백 종류의 물체 감지

* 심층신경망 훈련 중 발생하는 문제
    * 그레이디언트 소실/폭등: 심층신경망의 아래족으로 갈수록 그레이디언트가 점점 더 작아지거나 커지는 현상
    * 훈련 데이터 부족 또는 너무 비싼 레이블 작업
    * 극단적으로 느린 훈련과정
    * 과대적합: 수백만개의 파라미터에 의해 과대적합 가능성 매우 큼. 특히 훈련 샘플이 충분하지 않거나 잡음이 많은 경우 그러함.

## 주요 내용

* 언급된 문제 해결책 제시

## 그레이디언트 소실/폭등 문제

* (10장에서 설명한) 역전파 알고리즘: 출력층에서 입력층으로 오차 그레이디언트를 전파

* 하위층으로 갈 수록 그레이디언트 소실/폭등 문제 발생 자주 발생.
    * 그레이디언트 소실: 최적의 모델로 수렴하지 않음.
    * 그레이디언트 폭등: 알고리즘 발산.

* 원인: 활성화 함수와 가중치 초기화를 위해 아래 조합을 선택하였기 때문임.
    * 활성화 함수: 로지스틱 활성화 함수
    * 가중치 초기화: 표준정규분포 활용

<img src="images/ch11/homl11-01.png" width="400"/>

### 초기화 방식 선택

* 층에 사용하는 활성화 함수의 종류에 따라 아래 세 가지 초기화 방식 중 하나 선택

* 글로로(Glorot) 초기화 

* 르쿤(LeCun) 초기화

* 헤(He) 초기화

#### 글로로(Glorot) 초기화

* 팬-인/팬-아웃
    * fan-in(팬-인, $\textit{fan}_{\textrm{in}}$): 층에 들어오는 입력 수
    * fan-out(팬-아웃, $\textit{fan}_{\textrm{out}}$): 층에서 나가는 출력 수

$$
\textit{fan}_{\textrm{avg}} = \frac{\textit{fan}_{\textrm{in}} + \textit{fan}_{\textrm{out}}}{2}
$$

* 글로로 초기화(정규분포 활용)
    * 평균($\mu$) = $0$
    * 분산($\sigma^2$) = $\frac{1}{\textit{fan}_{\textrm{avg}}}$

* 글로로 초기화(균등분포 활용)
    * $-r$과 $r$ 사이의 균등분포
    
        $$r = \sqrt{\frac{3}{\textit{fan}_{\textrm{avg}}}}$$

#### 르쿤(LeCun) 초기화

* 글로로 초기화 정의에서 $\textit{fan}_{\textrm{avg}}$를 $\textit{fan}_{\textrm{in}}$ 으로 대체.

#### 헤(He) 초기화

* 정규분포 활용 초기화:

$$
\sigma^2 = \frac{2}{\textit{fan}_{\textrm{in}}}
$$

* 균등분포 활용 초기화:

$$
r = \sqrt{3 \sigma^2}
$$

#### 활성화 함수와 초기화 방식

* 층을 생성할 때 사용하는 활성화 함수에 따라 다른 가중치 초기화 방식을 사용해야 함.

|초기화 전략| 활성화 함수 | 정규분포 초기화 | 균등분포 초기화 |
| :--- | :--- | :--- | :--- |
| Glorot 초기화 | 활성화 함수 없는 경우, 하이퍼볼릭 탄젠트, 로지스틱, 소프트맥스 | glorot_normal | glorot_uniform |
| He 초기화 | ReLU 함수와 그 변종들 | he_normal | he_uniform |
| LeCun 초기화 | SELU | lecun_normal | lecun_uniform |

* 케라스의 기본 초기화 설정값: `glorot_uniform`

#### 예제

* 층을 만들 때 정규분포를 이용한 He 초기화를 사용하고자 하는 경우

```python
keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal")
```

#### 예제

* $\textit{fan}_{\textrm{in}}$ 대신 $\textit{fan}_{\textrm{out}}$ 기반의 균등분포 He 초기화를 사용하고자 할 경우
    * `VarianceScaling` 클래스 활용

```python
init = keras.initializers.VarianceScaling(scale=2., mode='fan_avg',
                                          distribution='uniform')
keras.layers.Dense(10, activation="relu", kernel_initializer=init)
```

### 활성화 함수 선택

* 심층신경망의 층에서 사용되는 활성화 함수는 시그모이드 함수보다 아래 함수들이 보다 좋은 성능을 발휘함.

* ReLU

* LeakyReLU

* RReLU

* PReLU

* ELU

* SELU

#### ReLU

* 2010년도에 소개됨.

* $\textrm{ReLU}_\alpha(z) = \max(0, z)$

* 완벽하지 않음

* 입력의 가중치 합이 음수가 되면 뉴런이 죽게 되어 경사하강법이 제대로 작동하지 않게됨.

<img src="images/ch11/homl11-02a.png" width="300"/>

#### LeakyReLU

* 2014년에 소개됨
* $\textrm{LeakyReLU}_\alpha(z) = \max(\alpha\, z, z)$
    * $\alpha$: 새어 나가는 정도를 결정하는 하이퍼파라미터

* ReLU 보다 좋은 성능 발휘
    * 기본값: $\alpha = 0.1$
    * $\alpha = 0.2$로 할 때 좀 더 성능 좋아짐.

<img src="images/ch11/homl11-03.png" width="350"/>

#### RReLU

* $\alpha$를 주어진 범위에서 무작위로 선택하는 LeakyReLU
* 꽤 잘 작동함
* 과대적합을 줄이는 규제역할도 수행하는 것으로 보임

#### PReLU

* 역전퐈 과정에서 $\alpha$값도 조정됨
* 대규모 이미지 데이터셋에서 ReLU 보다 성능 좋음.
* 소규모 데이터세에서는 과대적합 위험성 존재.

#### ELU

* 2015년도에 소개됨.
* 앞서 언급된 ReLU 변종들보다 성능 좋은 것으로 보임.
* 훈련 시간 줄어듦.

$$
\textrm{ELU}_\alpha(z) = 
\begin{cases}
\alpha(\exp(z) - 1) & \text{if } z < 0, \\
z & \text{if } z \ge 0.
\end{cases}
$$

<img src="images/ch11/homl11-04.png" width="350"/>

* ELU 함수의 장단점
    * 수렴 속도 빠름
    * 계산이 느림. 지수함수가 사용되기 때문.
    * 따라서 테스트할 때 ReLU를 사용하는 신경망보다 느림.

#### SELU

* 2017년도에 소개됨.
* 스케일이 조정된 ELU 활성화 함수의 변종

* 아래 조건 하에서 뛰어난 성능 발휘함

    * 입력특성이 표준화(평균 0, 표준편차 1)되어야 함.
    * 모든 은닉층에서 그쿤 정규분포 초기화 사용
    * 일렬로 쌓은 층만 사용해야 함.
    * 모든 층이 완전연결층이어야 함.

* 경우에 따라 합성곱 신경망에서도 좋은 성능 발휘

#### 심층신경망의 은닉층에 대한 활성화 함수 선택 가이드라인

* 일반적으로

    SELU > ELU > LeakyReLU 와 기타 변종들 > ReLU > 로지스틱

* 신경망이 자기정규화(self-normlizing)하지 않은 경우: SELU 보다 ELU 선호

* 시간과 컴퓨팅파워가 충분한 경우: 교차검증을 이용하여 여러 활성화함수 평가

* 실행속도가 중요한 경우: LeakyReLU

* 과대적합 발생하는 경우: RReLU 

* 훈련세트가 매우 큰 경우: PReLU

* 훈련속도가 중요한 경우: ReLU
    * 이유: 기존에 많은 라이브러리와 가속기가 개발되었기 때문.

#### 예제

* LeakyReLU 활성화 함수 사용

* LeakyReLU 층을 만들고 모델에서 적용하려는 층 뒤에 추가

```python
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(10, activation="softmax")
])
```

#### 예제

* PReLU 활성화 함수 사용

* PReLU 층을 만들고 모델에서 적용하려는 층 뒤에 추가

```python
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.PReLU(),
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.PReLU(),
    keras.layers.Dense(10, activation="softmax")
])
```

#### 예제

* RReLU 활성화 함수 사용

* 케라스에서 지원하지 않지만 간단하게 구현 가능. (12장 연습문제 참조)

#### 예제

* SELU 활성화 함수 사용

* 층을 만들 때 `activation="selu"` 와 `kernel_initializer="lecun_normal"` 지정.

```python
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(300, activation="selu",
                             kernel_initializer="lecun_normal"))
for layer in range(99):
    model.add(keras.layers.Dense(100, activation="selu",
                                 kernel_initializer="lecun_normal"))
model.add(keras.layers.Dense(10, activation="softmax"))
```

###  배치 정규화